In [1]:
!pip install gigachat
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from gigachat import GigaChat
from tqdm.notebook import tqdm

In [3]:
class GigaChatAPI:
    """
    API for accessing GigaChat
    List of available models: GigaChat(...).get_models()
    Get API key: https://developers.sber.ru/studio/login
    """

    def __init__(
        self, api_key: str, model: str = "GigaChat", verify_ssl_certs: bool = False
    ):
        self.api_key = api_key
        self.verify_ssl_certs = verify_ssl_certs
        self.model = model
        self.giga = GigaChat(
            credentials=self.api_key,
            verify_ssl_certs=self.verify_ssl_certs,
            model=self.model,
        )

    def prompt(self, prompt: str) -> str:
        response = self.giga.chat(prompt)
        return response.choices[0].message.content

    def switch_model(self, model: str) -> None:
        self.model = model
        self.giga = GigaChat(
            credentials=self.api_key,
            verify_ssl_certs=self.verify_ssl_certs,
            model=self.model,
        )

    def switch_system_prompt(self, system_prompt: str) -> None:
        self.system_prompt = system_prompt

In [4]:
data = load_dataset('RussianNLP/rucola', split = 'validation')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/7869 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2787 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2789 [00:00<?, ? examples/s]

In [5]:
giga_chat = GigaChatAPI(api_key="YWM4ZmE1YzUtM2E2MC00YzIzLTkxODktNjkwZDI4NDViNGM2OmVjZTQ0MzdmLTg1NjgtNDU1Zi04MzIyLTE0OGU5ZmM5MDlmNw==", verify_ssl_certs=False)
giga_chat.switch_system_prompt('Ты учитель русского языка. Твоя задача — помогать своим ученикам.')
#giga_chat.prompt("Где ошибка в предложении: Петеее надо купить фруктов?")

### label

In [6]:
bad_sentences = [11, 108, 329, 332, 355, 772, 1043, 1069, 1119, 1218, 1251, 1305,
                 1367, 1514, 1527, 1557, 1573, 1586, 1688, 1893, 1939,
                 2070, 2081, 2102, 2120, 2179, 2181, 2259,
                 2342, 2352, 2427, 2437, 2570, 2592, 2614, 2620, 2642, 2705, 2752, 2771]
bad_sentences.extend([355, 1025, 1075, 1110, 1139, 1156, 1174, 1185, 1342, 1361, 1384, 1464, 1502, 1585, 1630,
                      1644, 1652, 1742, 1797, 1216, 1252, 1517, 1600, 1719, 1828, 1856, 1911, 1935, 2206, 2247, 2498, 2628,
                                      1842, 1873, 2027, 2171, 2245, 2288, 2307, 2329, 2367, 2404, 2644, 2714, 2764])

In [10]:
i =  0
for message in tqdm(data['sentence']):
  if i in bad_sentences:
    pass
  else:
    a = giga_chat.prompt(f'Правильно ли следующее предложение с точки зрения грамматике русского языка: "{message}"? Ответь только ''да'' или ''нет''.')
    #print(a)
    if a.lower().startswith('да'):
      with open('gigachat_rucola_ru_18_05_2024_label_withrole_2.tsv', 'a') as f:
        f.write(str(i)+'\t'+'1'+'\n')
    else:
      with open('gigachat_rucola_ru_18_05_2024_label_withrole_2.tsv', 'a') as f:
        f.write(str(i)+'\t'+'0'+'\n')
  i += 1

  0%|          | 0/1 [00:00<?, ?it/s]

### label+category

In [8]:
import re
import json

In [11]:
i = 0
for message in tqdm(data['sentence']):
  if i in bad_sentences:
    pass
  else:
    a = giga_chat.prompt(f'''Есть ли ошибка в предложении: "{message}"?
    Если ошибки нет, ответь только "нет". Если ошибка есть – ответь да и укажи, к какому типу относится ошибка.
    Ошибка может относиться к одному из следующих типов: морфологическая, синтаксическая, семантическая или галлюцинация.
    Примеры ошибок: морфологическая ― "Увидал, что у санитара нет сапогов ― велел дать." (правильный вариант ― "Увидал, что у санитара нет сапог ― велел дать."),
    синтаксическая ― "Этим летом не никуда ездили." (правильный вариант ― "Этим летом никуда не ездили"),
    семантическая ― "Он живет в небольшом городке у Волги.",
    галлюцинация ― "Люк останавливает удачу от этого.".''')
    #print(a)
    if a.lower().startswith('нет') or (re.findall('нет .*ошибки|ошибки .*нет|нет .*ошибок|ошибок .*нет', a.lower()) != []
                                      or (re.findall('отсутству(е|ю)т', a.lower())) != []):
      with open('gigachat_rucola_ru_18_05_2024_label+category.tsv', 'a') as f:
        f.write(str(i)+'\t'+'0' + '\n')
    elif 'морфолог' in a:
      with open('gigachat_rucola_ru_18_05_2024_label+category.tsv', 'a') as f:
        f.write(str(i)+'\t'+'Morphology' + '\n')
    elif 'синтакси' in a:
      with open('gigachat_rucola_ru_18_05_2024_label+category.tsv', 'a') as f:
        f.write(str(i)+'\t'+'Syntax' + '\n')
    elif 'семанти' in a:
      with open('gigachat_rucola_ru_18_05_2024_label+category.tsv', 'a') as f:
        f.write(str(i)+'\t'+'Semantics' + '\n' )
    elif 'галлюцина' in a:
      with open('gigachat_rucola_ru_18_05_2024_label+category.tsv', 'a') as f:
        f.write(str(i)+'\t'+'Hallucination' + '\n')
    else:
      with open('gigachat_rucola_ru_18_05_2024_label+category.tsv', 'a') as f:
        f.write(str(i)+'\t'+a.upper() + '\n')


  i += 1

  0%|          | 0/1 [00:00<?, ?it/s]

### label+explanation

In [12]:
import re

In [13]:
for j in tqdm(range(len(data['sentence']))):
    if j in bad_sentences:
      pass
    else:
      a = giga_chat.prompt(f'''Есть ли ошибка в предложении: {data['sentence'][j]}.
Если ошибки нет, ответь только "нет". Если ошибка есть – ответь да и подробно объясни, в чем заключается ошибка.''')
      a = a.replace('\n', ' ')
      if a.lower().startswith('нет') or (re.findall('нет .*ошибки|ошибки .*нет|нет .*ошибок|ошибок .*нет', a.lower()) != []
      or (re.findall('ошибк.*отсутству(е|ю)т', a.lower())) != []) or (re.findall('отсутству(е|ю)т.*ошибк', a.lower())) != [] or 'предложение является правильным' in a:
        label = '1'
      else:
        label = '0'
      #print(j, a)
      if label == '0':
        with open('gigachat_rucola_ru_07_05_2024_label+explanation_4.tsv', 'a') as f:
          f.write(str(j) + '\t' + label + '\t' + data['sentence'][j] + '\t' + a + '\n')
      else:
        with open('gigachat_rucola_ru_07_05_2024_label+explanation_4.tsv', 'a') as f:
          f.write(str(j) + '\t' + label + '\t' + data['sentence'][j] + '\t' + 'OK' + '\n')

  0%|          | 0/1 [00:00<?, ?it/s]

### label+correction

In [17]:
import re

In [18]:
bad_sentences = [11, 108, 329, 772, 1043, 1069, 1119, 1218, 1251, 1305,
                 1367, 1514, 1527, 1557, 1573, 1586, 1688, 1893, 1939,
                 2070, 2081, 2102, 2120, 2179, 2181, 2259,
                 2342, 2352, 2427, 2437, 2570, 2592, 2620, 2705, 2752, 2771]
bad_sentences.extend([1110, 1139, 1174, 1185, 1585, 1644, 1652, 1742,
                                      1842, 1873, 2027, 2171, 2307, 2329, 2404])

In [16]:
for j in tqdm(range(len(data['sentence']))):
    if j in bad_sentences:
      pass
    else:
      a = giga_chat.prompt(f'''Дано предложение на русском языке, в котором, возможно, есть ошибка: {data['sentence'][j]}.
Если данное предложение неправильное, напиши "есть ошибка" и исправь предложение. Если данное предложение правильное, напиши "ошибок нет".
      ''')
      a = a.replace('\n', ' ')
      if "ошибок нет" in a.lower() or "нет ошибок" in a.lower():
        label = '1'
      else:
        label = '0'
      #print(j, a)
      if label == '0':
        with open('gigachat_rucola_ru_19_05_2024_label+correction_3.tsv', 'a') as f:
          f.write(str(j) + '\t'  + data['sentence'][j] + '\t' + label + '\t' + a + '\n')
      else:
        with open('gigachat_rucola_ru_19_05_2024_label+correction_3.tsv', 'a') as f:
          f.write(str(j) + '\t'  + data['sentence'][j] + '\t' + label + '\t' + 'OK' + '\n')

  0%|          | 0/1 [00:00<?, ?it/s]

### label+cat+expl

In [19]:
bad_sentences = [11, 54, 57, 108, 329, 335, 358, 609, 772, 1009, 1043, 1057, 1069, 1119, 1127, 1209, 1218, 1251, 1305, 1313,
                 1367, 1514, 1527, 1557, 1573, 1586, 1688, 1893, 1939,
                 2070, 2081, 2102, 2120, 2179, 2181, 2259,
                 2342, 2352, 2427, 2437, 2570, 2592, 2614, 2620, 2630, 2642, 2705, 2744, 2752, 2771]
bad_sentences.extend([1110, 1139, 1174, 1185, 1585, 1644, 1652, 1742,
1842, 1873, 2027, 2171, 2307, 2329, 2404, 453, 595, 1093, 1252, 1935, 2272, 2283])

In [20]:
import re

In [21]:
for j in tqdm(range(len(data['sentence']))):
    if j in bad_sentences:
      pass
    else:
      a = giga_chat.prompt(f'''Перед тобой предложение на русском языке, в котором, возможно, есть ошибка: {data['sentence'][j]}
Если в предложении есть ошибка, ответь, к какой категории она относится, и объясни, в чем заключается ошибка.
Ошибка может относиться к одной из следующих категорий: морфологическая, синтаксическая, семантическая или галлюцинации.
Примеры ошибочных предложений:
синтаксическая ошибка ― "Этим летом не никуда ездили.", семантическая ошибка ― "Он живет в небольшом городке у Волги.",
морфологическая ошибка ― "Увидал, что у санитара нет сапогов ― велел дать.",
галлюцинаци ― "Люк останавливает удачу от этого.". Напиши ответ в формате: категория ошибки:, объяснение: [твое объяснение].
Если предложение не содержит ошибок, ответь только "в предложении нет ошибок" и больше ничего.''')
      a = a.replace('\n', ' ')
      if re.findall('нет .*(ошибок|ошибки)', a.lower()) != [] or re.findall('(ошибок|ошибки) нет', a.lower()) != []:
        label = '1'
      else:

        label = '0'
        if 'морфологи' in a.lower():
          category = 'Morphology'
        elif 'синтакси' in a.lower():
          category = 'Syntax'
        elif 'семанти' in a.lower():
          category = 'Semantics'
        elif 'галлюцина' in a.lower():
          category = 'Hallucination'
        else:
          category = 'ERROR'
      #print(j, a)
      if label == '0':
        with open('gigachat_rucola_ru_19_05_2024_label+cat+expl_4.tsv', 'a') as f:
          f.write(str(j) + '\t'  + data['sentence'][j] + '\t' + label + '\t' + category + '\t' + a + '\n')
      else:
        with open('gigachat_rucola_ru_19_05_2024_label+cat+expl_4.tsv', 'a') as f:
          f.write(str(j) + '\t'  + data['sentence'][j] + '\t' + label + '\t' + '0' + '\t' + 'OK' + '\n')

  0%|          | 0/1 [00:00<?, ?it/s]

### label+cat+corr

In [ ]:
bad_sentences = [11, 54, 57, 108, 329, 335, 358, 609, 772, 1009, 1043, 1057, 1069, 1119, 1127, 1209, 1218, 1251, 1305, 1313,
                 1367, 1514, 1527, 1557, 1573, 1586, 1688, 1893, 1939,
                 2070, 2081, 2102, 2120, 2179, 2181, 2259,
                 2342, 2352, 2427, 2437, 2570, 2592, 2614, 2620, 2630, 2642, 2705, 2744, 2752, 2771]
bad_sentences.extend([1110, 1139, 1174, 1185, 1585, 1644, 1652, 1742,
1842, 1873, 2027, 2171, 2307, 2329, 2404, 453, 595, 1093, 1252, 1935, 2272, 2283])

import re

In [22]:
for j in tqdm(range(len(data['sentence']))):
    if j in bad_sentences:
      pass
    else:
      a = giga_chat.prompt(f'''Перед тобой предложение на русском языке, в котором, возможно, есть ошибка: {data['sentence'][j]}.
Если в предложении нет ошибок, ответь только "нет ошибок".
Если в предложении есть ошибка, ответь, к какой категории она относится: морфологическая, синтаксическая, семантическая или галлюцинация.
Также напиши, как можно исправить предложение. В этом случае напиши ответ в формате: "категория: [категория ошибки], правильный вариант: [правильный вариант]".
Примеры ошибочных предложений:
синтаксическая ошибка ― "Этим летом не никуда ездили.", семантическая ошибка ― "Он живет в небольшом городке у Волги.",
морфологическая ошибка ― "Увидал, что у санитара нет сапогов ― велел дать.",
галлюцинация ― "Люк останавливает удачу от этого.".
''')

      a = a.replace('\n', ' ')
      if (re.findall('нет .*(ошибок|ошибки)', a.lower()) != [] or re.findall('(ошибок|ошибки) нет', a.lower()) != []
or (re.findall('ошибк(а|и) отсутству(е|ю)т', a.lower()) != []) or 'не обнаружен' in a.lower()) or a.lower() == data['sentence'][j]:
        label = '1'
      else:
        label = '0'
        corr = a[a.find('вариант')+8:]
        corr = corr.lstrip(' ')

        if 'морфологи' in a.lower():
          category = 'Morphology'
        elif 'синтакси' in a.lower():
          category = 'Syntax'
        elif 'семанти' in a.lower():
          category = 'Semantics'
        elif 'галлюцина' in a.lower():
          category = 'Hallucination'
        else:
          category = 'ERROR'

      #print(j, a)

      if label == '0':
        with open('gigachat_rucola_ru_20_05_2024_label+cat+corr_2.tsv', 'a') as f:
          f.write(str(j) + '\t'  + data['sentence'][j] + '\t' + label + '\t' + category + '\t' + corr + '\n')
      else:
        with open('gigachat_rucola_ru_20_05_2024_label+cat+corr_2.tsv', 'a') as f:
          f.write(str(j) + '\t'  + data['sentence'][j] + '\t' + label + '\t' + '0' + '\t' + 'OK' + '\n')

  0%|          | 0/1 [00:00<?, ?it/s]

### label+expl+corr

In [23]:
for j in tqdm(range(len(data['sentence']))):
    if j in bad_sentences:
      pass
    else:
      a = giga_chat.prompt(f'''Перед тобой предложение на русском языке, в котором, возможно, есть ошибка: {data['sentence'][j]}
Если в предложении есть ошибка, подробно объясни, в чем именно заключается ошибка, и исправь предложение.
Если ты нашел ошибку, напиши ответ в формате: "объяснение: [твое объяснение], исправленное предложение: [правильное предложение]".
Если предложение не содержит ошибок, ответь только "в предложении нет ошибок".
Пожалуйста, если ты нашел ошибку, опиши ее максимально подробно, чтобы помочь своему ученику.
'''
)
      a = a.replace('\n', ' ')
      if (re.findall('нет .*(ошибок|ошибки)', a.lower()) != [] or re.findall('(ошибок|ошибки) нет', a.lower()) != []
or (re.findall('ошибк(а|и) отсутству(е|ю)т', a.lower()) != []) or 'не обнаружен' in a.lower()):
        label = '1'
      else:
        label = '0'



      if label == '0':
        #expl = a[a.find('бъяснение:')+11:a.find('справленное предложение:')-2]
        corr = a[a.find('справленное предложение:')+25:]
        with open('gigachat_rucola_ru_20_05_2024_label+expl+corr_4.tsv', 'a') as f:
          f.write(str(j) + '\t'  + data['sentence'][j] + '\t' + label +   '\t' + a + '\t' +corr + '\n')
      else:
        with open('gigachat_rucola_ru_20_05_2024_label+expl+corr_4.tsv', 'a') as f:
          f.write(str(j) + '\t'  + data['sentence'][j] + '\t' + label +  '\t' + 'OK' + '\t' + 'OK' + '\n')

  0%|          | 0/1 [00:00<?, ?it/s]

### label+cat+expl+corr

In [24]:
for j in tqdm(range(len(data['sentence']))):
    if j in bad_sentences:
      pass
    else:
      a = giga_chat.prompt(f'''Перед тобой предложение на русском языке, в котором, возможно, есть ошибка: {data['sentence'][j]}.
Если в предложении нет ошибок, ответь только "в предложении нет ошибок".
Если в предложении есть ошибка, напиши ответ в следующем формате: "категория: [категория ошибки], объяснение: [твое объяснение], исправленное предложение: [правильное предложение]"
Ошибка может относиться к одной из следующих категорий: морфологическая, синтаксическая, семантическая или галлюцинация. Пожалуйста, объясняй ошибку подробно.
'''
)
      a = a.replace('\n', ' ')
      if (re.findall('нет .*(ошибок|ошибки)', a.lower()) != [] or re.findall('(ошибок|ошибки) нет', a.lower()) != []
or (re.findall('ошибк(а|и) отсутству(е|ю)т', a.lower()) != []) or 'не обнаружен' in a.lower()):
        label = '1'
      else:
        label = '0'


      if label == '0':
        if 'морфологи' in a.lower():
          category = 'Morphology'
        elif 'синтакси' in a.lower():
          category = 'Syntax'
        elif 'семанти' in a.lower():
          category = 'Semantics'
        elif 'галлюцина' in a.lower():
          category = 'Hallucination'


        #expl = a[a.find('бъяснение:')+11:a.find('справленное предложение:')-2]
        corr = a[a.find('справленное предложение:')+25:]
        with open('gigachat_rucola_ru_20_05_2024_label+cat+expl+corr_4.tsv', 'a') as f:
          f.write(str(j) + '\t'  + data['sentence'][j] + '\t' + label +  '\t' + category + '\t' + a + '\t' +corr + '\n')
      else:
        with open('gigachat_rucola_ru_20_05_2024_label+cat+expl+corr_4.tsv', 'a') as f:
          f.write(str(j) + '\t'  + data['sentence'][j] + '\t' + label + '\t' + '0' + '\t' + 'OK' + '\t' + 'OK' + '\n')

  0%|          | 0/1 [00:00<?, ?it/s]